In [1]:
%cd ../..
%run cryptolytic/notebooks/init.ipynb
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import tensorflow as tf
import cryptolytic.util as util
import cryptolytic.start as start
import cryptolytic.viz.plot as plot
import cryptolytic.data.sql as sql
import cryptolytic.data as d
from cryptolytic.util import *
import cryptolytic.data.historical as h
import cryptolytic.model as m
import cryptolytic.model.lstm_framework as lstm
import cryptolytic.model.model_framework as mfw
import cryptolytic.model.data_work as dw
import cryptolytic.model.hyperparameter as hyper
import cryptolytic.model.cron_model as cron_model

from tensorflow.keras.layers import Dense, Embedding, Conv1D, Activation,\
    Add, Input, LSTM, Reshape
from tensorflow.keras.initializers import glorot_uniform, glorot_uniform


from matplotlib.pylab import rcParams
from IPython.core.display import HTML
from pandas.plotting import register_matplotlib_converters # to stop a warning message
import cryptolytic.model.model_framework as mf

ohclv = ['open', 'high', 'close', 'low', 'volume']
plt.style.use('ggplot')
rcParams['figure.figsize'] = 20,7
start.init()
register_matplotlib_converters()


# Make math readable
HTML("""
<style>
.MathJax {
    font-size: 2rem;
}
</style>""")

/home/me/Documents/Git/Lambda-School-Labs/cryptolytic-ds
Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# LSTNet
- Convolutional Layer
- Reccurent and Reccurent Skip Layer
- Fully Connectedand Element Wise Sum Output -> Prediction layer
- Autoregressive Highway -> Prediction layer

$z_l = w_l * a_{l-1} + b_l$


$a_l = g_l(z_l)$

cache z_l (useful for propagation step)

In [2]:
bdir(mfw)

['Activation',
 'Add',
 'Conv1D',
 'Dense',
 'Embedding',
 'Input',
 'LSTM',
 'MinMaxScaler',
 'Model',
 'Sequential',
 'TAN',
 'adict',
 'bdir',
 'compose',
 'constraints',
 'conv_block',
 'create_framework_lstm',
 'create_model',
 'd_block',
 'date',
 'dict_matches',
 'filterl',
 'first',
 'fit_model',
 'fit_stacked_model',
 'functools',
 'glorot_uniform',
 'identity_block',
 'layers',
 'load_all_models',
 'losses',
 'mapl',
 'model_compile',
 'models',
 'mpl',
 'multiprocessing',
 'np',
 'os',
 'pd',
 'plt',
 'pprint',
 'predictions',
 'print_function',
 'regularizers',
 'save_model',
 'scipy',
 'select_keys',
 'sequence',
 'signal',
 'tan_layer',
 'tf',
 'timeout',
 'timeout_handler']

In [3]:
mfw.d_block

<function cryptolytic.model.model_framework.d_block(X)>

In [7]:
import tensorflow.keras.layers as layers
layers.Conv1D(5, )

TypeError: __init__() missing 1 required positional argument: 'kernel_size'

In [8]:
help(layers.Conv1D)

Help on class Conv1D in module tensorflow.python.keras.layers.convolutional:

class Conv1D(Conv)
 |  Conv1D(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
 |  
 |  1D convolution layer (e.g. temporal convolution).
 |  
 |  This layer creates a convolution kernel that is convolved
 |  with the layer input over a single spatial (or temporal) dimension
 |  to produce a tensor of outputs.
 |  If `use_bias` is True, a bias vector is created and added to the outputs.
 |  Finally, if `activation` is not `None`,
 |  it is applied to the outputs as well.
 |  
 |  When using this layer as the first layer in a model,
 |  provide an `input_shape` argument
 |  (tuple of integers or `None`, e.g.
 |  `(10, 128)` for sequences of 1

In [11]:
import os
history_size = 400
input_len = 8000
lahead = 12*3
step = 2
period = 300
to_drop = lahead - 1
input_len = input_len + to_drop
batch_size = 200

df_orig = None
df_orig = d.get_df({'start':'06-01-2019', 'period':period, 'trading_pair':'btc_usd', 'exchange_id':'bitfinex'},
               n=input_len)

import ta
df = df_orig
df = df.sort_index()
df = df._get_numeric_data().drop(["period"], axis=1, errors='ignore')
df = df.filter(regex="(?!timestamp_.*)", axis=1) # filter out timestapm_ metrics
df = ta.add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume").dropna(axis=1)
df_diff = (df - df.shift(1, fill_value=0)).rename(lambda x: x+'_diff', axis=1)
df = pd.concat([df, df_diff], axis=1)
dataset = dw.normalize(df.values)
target = df.columns.get_loc('close') 
y = dataset[:, target]
history = {'loss' : [], 'val_loss' : []}
x_train, y_train, x_val, y_val = dw.windowed(dataset, target, batch_size, history_size, step, lahead)

df                           api  exchange trading_pair   timestamp  period  \
datetime                                                                   
2019-01-06 06:00:00  bitfinex  bitfinex      btc_usd  1546754400     300   
2019-01-06 06:05:00  bitfinex  bitfinex      btc_usd  1546754700     300   
2019-01-06 06:10:00  bitfinex  bitfinex      btc_usd  1546755000     300   
2019-01-06 06:15:00  bitfinex  bitfinex      btc_usd  1546755300     300   
2019-01-06 06:20:00  bitfinex  bitfinex      btc_usd  1546755600     300   
...                       ...       ...          ...         ...     ...   
2019-02-03 10:15:00  bitfinex  bitfinex      btc_usd  1549188900     300   
2019-02-03 10:20:00  bitfinex  bitfinex      btc_usd  1549189200     300   
2019-02-03 10:25:00  bitfinex  bitfinex      btc_usd  1549189500     300   
2019-02-03 10:30:00  bitfinex  bitfinex      btc_usd  1549189800     300   
2019-02-03 10:35:00  bitfinex  bitfinex      btc_usd  1549190100     300   

        

/home/me/anaconda3/lib/python3.7/site-packages/ta/trend.py:543: RuntimeWarning:

invalid value encountered in double_scalars

/home/me/anaconda3/lib/python3.7/site-packages/ta/trend.py:547: RuntimeWarning:

invalid value encountered in double_scalars



In [18]:
def create_model(x_train, params, batch_size, lahead):
    input_shape = x_train.shape[-2:]
    X_input = Input(input_shape, batch_size=batch_size)
    X = X_input
    X = mfw.tan_block(X)
    model = Model(inputs=X_input, outputs=X, name='model1')
    model.compile(loss='mse',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001))
    return model

In [45]:
params = adict()
model = create_model(x_train, params, batch_size, lahead)

In [16]:
mfw.fit_model(model, x_train, y_train, x_val, y_val, batch_size)

AttributeError: 'NoneType' object has no attribute 'fit'

# Regularizing with structured signals
- Batch of labeled samples with neighbors
- Each training sample is augmented to include its neighbor information from a given structure.
- Get a new training batch where both the training samples and their neighbors are included 
- Next both the training sample and its neighbors are fed into the neural net
-> Input Layer
    a) Sample Features -> 
    b) Neighbor Features
    
Neighbor Embedding
$\phi(x_u)$

Sample Embedding 
$\phi(x_u)$

Regularization -> Supervised Loss -> Sample labels

Then, the difference between a sample embedding and a neighbor embedding is calculated and added into the final loss as a regularization term. By adding a regualization term, the network learns to keep the similarity between a point and its neighbor